## Style block (delete header)

[Einhverjar góðar upplýsingar um page-styling](https://stackoverflow.com/questions/3341485/how-to-make-a-html-page-in-a4-paper-size-pages)

<style type="text/css">
* {
    max-width: 800px;
}
</style>

<hr>

# Verkefni II

### Motion Control in Computer-Aided Modeling

##### Reality Check 5, Numerical Analysis 2nd ed., Sauer.
<hr>

#### Erling Óskar Kristjánsson, eok4@hi.is

#### Davíð Freyr Björnsson, dfb2@hi.is
<hr>

##### Námskeið: Töluleg Greining (Stæ405G)

##### Stofnun: Háskóli Íslands

##### Umsjónamaður: Sigurður Freyr Hafstein

##### Vor 2019
<hr>

## Introduction 

Computer-aided modeling and manufacturing requires precise control of spatial position
along a prescribed motion path. We will illustrate the use of Adaptive Quadrature to solve
a fundamental piece of the problem: equipartition, or the division of an arbitrary path into
equal-length subpaths.

In numerical machining problems, it is preferable to maintain constant speed along the
path. During each second, progress should be made along an equal length of the machine–
material interface. In other motion planning applications, including computer animation,
more complicated progress curves may be required: A hand reaching for a doorknob might 
begin and end with low velocity and have higher velocity in between. Robotics and virtual
reality applications require the construction of parametrized curves and surfaces to be
navigated. Building a table of small equal increments in path distance is often a necessary
first step.

Assume that a parametric path P = $\{x(t),\ y(t) \ | \  0 \leq t \leq 1\}$ is given. Figure 5.6 shows
the example path

$$
P =
\begin{cases}
x(t) = 0.5 + 0.3t + 3.9t^2 − 4.7t^3, \\
y(t) = 1.5 + 0.3t + 0.9t^2 − 2.7t^3
\end{cases}
$$

Bézier curve defined by the four points $(0.5,1.5)$, $(0.6,1.6)$, $(2,2)$, $(0,0)$. 
Points defined by evenly spaced parameter values $t = 0, 1/4, 1/2, 3/4, 1$ are shown. 

![(May need to host pictures like this online and supply a https link) Figure 5.6](img/Figure5_6.JPG)

Note that even spacing in parameter does not imply even spacing in arc length. 
Your goal is to apply quadrature methods to divide this path into $n$ equal lengths.

Recall from calculus that the arc length of the path from $t_1$ to $t_2$ is

$$
\int_{t_1}^{t_2} \sqrt{x'(t)^2+y'(t)^2}dt
$$

Only rarely does the integral yield a closed-form expression, and normally an Adaptive
Quadrature technique is used to control the parametrization of the path.

<hr>

## Suggested Activity 1

Write a Python function that uses Adaptive Quadrature to compute the arc length from $t_1 = 0$ to $t_2 = T$ for a given $T \leq 1$.

** Ready as far as $x'(t)$ and $y'(t)$ are supplied. The given $P(x(t),y(t))$ was differentiated manually. **

In [20]:
from functionsV2 import *
import numpy as np
from time import perf_counter

# Temporary global tol
tol = 0.5e-6
# Temporary global number of decimals
dec = 5

""" Suggested Activity 1 """
# Given parametric path with
# x(t) = 0.5 + 0.3*t + 3.9*t**2 - 4.7*t**3 and 
# y(t) = 1.5 + 0.3*t + 0.9*t**2 - 2.7*t**3
# We can manually compute the derivatives
dxdt = lambda t: 0.3 + 7.8*t - 14.1*t**2
dydt = lambda t: 0.3 + 1.8*t - 8.1*t**2

# Compute the lambda function that
# is to be integrgated
f = sqrtFunSquared(dxdt, dydt)

# Compute the corresponding arc length
# by computing the integral of f from
# 0 to 1 using the method of Adaptive Quadrature
arcLength = compArcLength(f, 0.0, 1.0, adQuad, tol)

print("The arc length is", round(arcLength, dec))

The arc length is 2.49525


<hr>

## Suggested Activity 2

Write a program that, for any input $s$ between $0$ and $1$, finds the parameter $t^∗(s)$
that is $s$ of the way along the curve. In other words, the arc length from $t = 0$
to $t = t^∗(s)$ divided by the arc length from $t = 0$ to $t = 1$ should be equal to $s$.

**In other other words: **

$$ s = \frac{\int_{0}^{t^*(s)} \sqrt{x'(t)^2+y'(t)^2}dt}{ \int_{0}^{1} \sqrt{x'(t)^2+y'(t)^2}dt}$$

Use the Bisection Method to locate the point $t^∗(s)$ to three correct decimal places. 

1. What function is being set to zero?<br>
$$ s \int_{0}^{1} \sqrt{x'(t)^2+y'(t)^2}dt - \int_{0}^{t^*(s)} \sqrt{x'(t)^2+y'(t)^2}dt = 0$$
2. What bracketing interval should be used to start the Bisection Method?<br>
The bracket interval [0, 1], since for $t^*(s) = 0$ it follows that $s=0$ and when $t^*(s) = 1$ it follows that $s=1$. So this range of $t^*(s)$ covers all the possible values of $s$.

In [21]:
# Let's time the function tStarOfS
# for s = 0.5 
s = 0.5
start = perf_counter()
tStar2 = tStarOfSBisect(f, s, adQuad, tol)
end = perf_counter()
elapsedTime2 = end - start

print('The optimal value of t for s =', round(s, dec), 'is', round(tStar2, dec))
print('and was computed in', round(elapsedTime2, dec), 'seconds')

# Let's verify if tStar really is the
# root of the function 
print('Verified:', (np.abs(compArcLength(f, 0.0, tStar2, adQuad, tol) / 
                           compArcLength(f, 0.0, 1.0, adQuad, tol) - s) < 0.001))

The optimal value of t for s = 0.5 is 0.80059
and was computed in 1.75286 seconds
Verified: True


<hr>

## Suggested Activity 3

Equipartition (_split into parts of equal length_) the path of Figure 5.6 into $n$ subpaths of equal length, for $n = 4$ and $n = 20$. Plot analogues of Figure 5.6, showing the equipartitions. If your computations are too slow, consider speeding up the Adaptive Quadrature with Simpson’s Rule, as suggested in Computer Problem 5.4.2.

We use the methodology from the previous example. First we implement an efficient version of the Adaptive Quadrature with Simpson's Rule, shown below.

Let us consider the scenario where $n = 4$: 
- We know that the length of the curve is approximately $2.49524674$, so in this scenario we want each part to be of length $0.623811685$.

In [22]:
# For n = 4
sArray = [0.0, 0.25, 0.5, 0.75, 1.0]

start = perf_counter()
tStarArray = [tStarOfSBisect(f, s, adQuadSimpson, tol) for s in sArray]
end = perf_counter()
elapsedTime3n4 = end - start
print("For n = 4 the computation of t*(s) took",
      round(elapsedTime3n4, dec), "seconds")

for i in range(len(sArray)-1):
    arclength = compArcLength(f, tStarArray[i], tStarArray[i+1], adQuadSimpson, tol)
    print('Arc length from', round(tStarArray[i], dec), end=' ')
    print('to', round(tStarArray[i+1], dec), end=' ') 
    print('is', round(arclength, dec))
    print('Proportional arc length:', end=' ')
    print(round(np.abs(arclength / compArcLength(f, 0.0, 1.0, adQuadSimpson, tol)), dec))

For n = 4 the computation of t*(s) took 0.10363 seconds
Arc length from 0.0 to 0.54587 is 0.62381
Proportional arc length: 0.25
Arc length from 0.54587 to 0.80059 is 0.62381
Proportional arc length: 0.25
Arc length from 0.80059 to 0.91683 is 0.62382
Proportional arc length: 0.25
Arc length from 0.91683 to 1.0 is 0.6238
Proportional arc length: 0.25


Let us then consider the scenario where $n = 20$: 
- We know that the length of the curve is approximately $2.49524674$, so in this scenario we want each part to be of length $0.124762337$.

In [23]:
# For n = 20
sArray = np.arange(0.00, 1.05, 0.05)
start = perf_counter()
tStarArray = [tStarOfSBisect(f, s, adQuadSimpson, tol) for s in sArray]
end = perf_counter()
elapsedTime3n20 = end - start
print("For n = 20 the computation of t*(s) program took", 
      round(elapsedTime3n20, dec), "seconds")
 
for i in range(len(sArray)-1):
    arclength = compArcLength(f, tStarArray[i], tStarArray[i+1], adQuadSimpson, tol)
    print('Arclength from', round(tStarArray[i], dec), end='')
    print(' to', round(tStarArray[i+1], dec), end=' ') 
    print('is:', round(arclength, dec))
    print('Proportional Arclength :', round(np.abs(arclength / 
                           compArcLength(f, 0.0, 1.0, adQuadSimpson, tol)), 2))

For n = 20 the computation of t*(s) program took 0.35944 seconds
Arclength from 0.0 to 0.14534 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.14534 to 0.24023 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.24023 to 0.33078 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.33078 to 0.43168 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.43168 to 0.54587 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.54587 to 0.63095 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.63095 to 0.68872 is: 0.12478
Proportional Arclength : 0.05
Arclength from 0.68872 to 0.7329 is: 0.12475
Proportional Arclength : 0.05
Arclength from 0.7329 to 0.76929 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.76929 to 0.80059 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.80059 to 0.82828 is: 0.12477
Proportional Arclength : 0.05
Arclength from 0.82828 to 0.85324 is: 0.12476
Proportional Arclength : 0.05
Arclength from 0.85324 to 0.8

<hr>

## Suggested Activity 4

Replace the Bisection Method in Step 2 with Newton’s Method, and repeat Steps 2 and 3.

1. What is the derivative needed? <br>
The derivative with respect to $t$ of the function that is being set to zero, namely $$ d \bigg( s \int_{0}^{1} \sqrt{x'(t)^2+y'(t)^2}dt - \int_{0}^{t^*(s)} \sqrt{x'(t)^2+y'(t)^2}dt \bigg) dt^{-1} $$
The derivative can be estimated in the necessary points using Python's `Autograd`. However, a loop of trials revealed that the Three-point centered-difference formula described in Chapter 5.1 gave the same estimates as `Autograd` to at least five decimal places, so that will be used instead.
2. What is a good choice for the initial guess? <br>
As in Suggested Activity 2, it is clear that the root lies in the interval $[0,1]$. However, more precision is possible when the  path is being equipartitioned into $n$ parts. When diving the path into $n$ paths, we use the initial guess of `tGuess=0` for the first $t^*(s)$, and then we use that $t^*(s)$ as an initial guess for the next $t^*(s)$, and so on.

3. Is computation time decreased by this replacement? <br>
As evinced in the subsequent code sniplet, the computation time is decreased by replacing the Bisection method with Newton's method.

Since the computed value of $t^*(s)$ for $s=0.5$ is approximately
0.8, a good initial guess when using Newton's method is 0.8

In [24]:
xGuess = 0.8
start = perf_counter()
tStar4 = tStarOfSNewton(f, s, adQuad, xGuess, tol)
end = perf_counter()
elapsedTime4 = end - start

print('The computed value of t*(s) for s =', s, end=' ')
print('using the Newton\'s method is')
print(round(tStar4, dec+3))

print('The computed value of t*(s) for s =', s, end=' ')
print('using the Bisection method is')
print(round(tStar2, dec+3), "\n")

print('Time required to compute t*(s) using the Newton\'s method is:')
print(round(elapsedTime4, dec+2), "seconds")
print('Time required to compute t*(s) using the Bisection method is:')
print(round(elapsedTime2, dec+2), "seconds")

The computed value of t*(s) for s = 0.5 using the Newton's method is
0.80059377
The computed value of t*(s) for s = 0.5 using the Bisection method is
0.80059338 

Time required to compute t*(s) using the Newton's method is:
1.0969371 seconds
Time required to compute t*(s) using the Bisection method is:
1.7528629 seconds


So we can see that the computed values for t*(s) are equal 
up to five decimals but Newton's method converges more quickly,
as expected.